1. A). 

$\delta^2_x u^n_{l,m} = u^n_{l-1,m}-2u^n_{l,m}+u^n_{l+1,m}$

$\delta^2_y u^n_{l,m} = u^n_{l,m-1}-2u^n_{l,m}+u^n_{l,m+1}$

$u^*_{l,m} = u^n_{l,m}+ \frac{1}{2}[\alpha_x\delta^2_x u^*_{l,m} + \alpha_y\delta^2_y u^n_{l,m}]$

$u^{n+1}_{l,m} = u^*_{l,m}+ \frac{1}{2}[\alpha_x\delta^2_x u^*_{l,m} + \alpha_y\delta^2_y u^{n+1}_{l,m}]$

$u(0,x,y) = sin(\pi x)sin(\pi y)$

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [11]:
def thomas_algorithm(a,b,c,d):
    n = len(d)
    c_star = np.zeros(n-1)
    d_star = np.zeros(n)
    
    c_star[0] = c[0] / b[0]
    d_star[0] = d[0] / b[0]
    
    for i in range(1, n-1):
        temp = b[i] - a[i-1] * c_star[i-1]
        c_star[i] = c[i] / temp
        d_star[i] = (d[i] - a[i-1] * d_star[i-1]) / temp
    
    d_star[-1] = (d[-1] - a[-1] * d_star[-2]) / (b[-1] - a[-1] * c_star[-2])
    
    x = np.zeros(n)
    x[-1] = d_star[-1]
    
    for i in range(n-2, -1, -1):
        x[i] = d_star[i] - c_star[i] * x[i+1]
    
    return x

In [107]:
def ADI_Method(Mx, My, N, T, x_length, y_length):
    dt = T / N
    dx = x_length / Mx
    dy = y_length / My
    alpha_x = dt / dx**2
    alpha_y = dt / dy**2

    u = np.zeros((N + 1, Mx + 1, My + 1))
    u_star = np.zeros((N + 1, Mx + 1, My + 1))
    x = np.linspace(0, x_length, Mx + 1)
    y = np.linspace(0, y_length, My + 1)

    for i in range(My + 1):
        for j in range(Mx + 1):
            if i == 0 or j == 0 or i == My or j == Mx:
                u[0, j, i] = 0
                u_star[0, j, i] = 0
            else:
                u[0, j, i] = np.sin(np.pi * x[j]) * np.sin(np.pi * y[i])
                u_star[0, j, i] = np.sin(np.pi * x[j]) * np.sin(np.pi * y[i])

    for n in range(N):  
        a = (-alpha_x / 2) * np.ones(Mx-2)
        b = (1 + alpha_x) * np.ones(Mx-1)
        c = (-alpha_x / 2) * np.ones(Mx-2)

        alphas = np.zeros((My - 1, My - 1))
        for z in range(len(alphas)):
            alphas[z,z] = (1 - alpha_y)
            if z != 0:
                alphas[z  - 1, z] = (alpha_y / 2)
            if z != (len(alphas) - 1):
                alphas[z + 1, z] = (alpha_y / 2)

        for j in range(My - 1, 0, -1):
            prev_u = np.dot(alphas, u[n, 1:-1, My - j])
            u_star[n+1,j, 1:-1] = thomas_algorithm(a, b, c, prev_u)

        a2 = (-alpha_y / 2) * np.ones(My-2)
        b2 = (1 + alpha_y) * np.ones(My-1)
        c2 = (-alpha_y / 2) * np.ones(My-2)

        alphas2 = np.zeros((Mx - 1, Mx - 1))
        for z2 in range(len(alphas2)):
            alphas2[z2,z2] = (1 - alpha_x)
            if z2 != 0:
                alphas2[z2  - 1, z2] = (alpha_x / 2)
            if z2 != (len(alphas2) - 1):
                alphas2[z2 + 1, z2] = (alpha_x / 2)

        for j2 in range(Mx - 1, 0, -1):
            prev_u_star = np.dot(alphas, u_star[n+1, j2, 1:-1])
            u[n+1,1:-1, j2] = thomas_algorithm(a2, b2, c2, prev_u_star)

    return x, y, u, alpha_x, alpha_y

In [140]:
from matplotlib import cm

fig = plt.figure(figsize=plt.figaspect(0.5))

x, y, u, alpha_x, alpha_y = ADI_Method(50, 50, 10000, 0.5, 1, 1)
X, Y = np.meshgrid(x, y)
print(alpha_x)

ax = fig.add_subplot(2, 2, 1, projection='3d')
ax.plot_surface(X, Y, u[0, :, :], cmap=cm.coolwarm, linewidth=0, antialiased=False)
ax.view_init(20, -50)
ax.set_xlabel('x')
ax.set_ylabel('t')
ax.set_zlabel('u')
ax.set_zlim(0, 1)

ax = fig.add_subplot(2, 2, 2, projection='3d')
ax.plot_surface(X, Y, u[100, :, :], cmap=cm.coolwarm, linewidth=0, antialiased=False)
ax.view_init(20, -50)
ax.set_xlabel('x')
ax.set_ylabel('t')
ax.set_zlabel('u')
ax.set_zlim(0, 1)

ax = fig.add_subplot(2, 2, 3, projection='3d')
ax.plot_surface(X, Y, u[500, :, :], cmap=cm.coolwarm, linewidth=0, antialiased=False)
ax.view_init(20, -50)
ax.set_xlabel('x')
ax.set_ylabel('t')
ax.set_zlabel('u')
ax.set_zlim(0, 1)

ax = fig.add_subplot(2, 2, 4, projection='3d')
ax.plot_surface(X, Y, u[1000, :, :], cmap=cm.coolwarm, linewidth=0, antialiased=False)
ax.view_init(20, -50)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('u')
ax.set_zlim(0, 1)

KeyboardInterrupt: 

<Figure size 960x480 with 0 Axes>

In [106]:
u = np.zeros((4 + 1, 4 + 1, 4 + 1))
x = np.linspace(0, 1, 4 + 1)
y = np.linspace(0, 1, 4 + 1)

for i in range(4 + 1):
    for j in range(4 + 1):
        if i == 0 or j == 0 or i == 4 or j == 4:
            u[0, j, i] = 0
        else:
            u[0, j, i] = np.sin(np.pi * x[j]) * np.sin(np.pi * y[i])

a = (-1 / 2) * np.ones(4-2)
b = (1 + 1) * np.ones(4-1)
c = (-1 / 2) * np.ones(4-2)

alphas = np.zeros((3, 3))
for x in range(len(alphas)):
    alphas[x,x] = (1- 2)
    if x != 0:
        alphas[x  - 1, x] = (1 / 2)
    if x != (len(alphas) - 1):
        alphas[x + 1, x] = (1 / 2)

for n in range(4):
    for j in range(4 - 1, 0, -1):
        prev_u = np.dot(alphas, u[n, j, 1:-1])
        u[n+1,1:-1, j] = thomas_algorithm(a, b, c, prev_u)

print(u)
print(u[1,1:-1, 1])

[[[ 0.          0.          0.          0.          0.        ]
  [ 0.          0.5         0.70710678  0.5         0.        ]
  [ 0.          0.70710678  1.          0.70710678  0.        ]
  [ 0.          0.5         0.70710678  0.5         0.        ]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 0.          0.          0.          0.          0.        ]
  [ 0.         -0.1132369  -0.16014116 -0.1132369   0.        ]
  [ 0.         -0.16005437 -0.22635107 -0.16005437  0.        ]
  [ 0.         -0.11276704 -0.15947667 -0.11276704  0.        ]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 0.          0.          0.          0.          0.        ]
  [ 0.          0.02564519  0.03624812  0.02553878  0.        ]
  [ 0.          0.03624812  0.05123481  0.03609771  0.        ]
  [ 0.          0.02553878  0.03609771  0.02543281  0.        ]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 0.          0.          0.    